In [1]:
import os

%pwd

'/home/jafarian/PycharmProjects/WIP-SSVEPA/research'

In [2]:
os.chdir("../")
%pwd

'/home/jafarian/PycharmProjects/WIP-SSVEPA'

In [3]:
# Entity Data Ingestion Development

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [4]:
from SSVEPA.constants import SCHEMA_FILE_PATH, CONFIG_FILE_PATH, PARAMS_FILE_PATH
from SSVEPA.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from SSVEPA import logger
from SSVEPA.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-09 23:47:31,736: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-09 23:47:31,737: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 23:47:31,739: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 23:47:31,740: INFO: common: created directory at: artifacts]
[2024-07-09 23:47:31,741: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-09 23:47:32,483: INFO: 2771580160: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D7C9:34BD36:12287FC:12F8986:668DA1E4
Accept-Ranges: bytes
Date: Tue, 

In [11]:
import os
import urllib.request as request
import zipfile
from SSVEPA import logger
from pathlib import Path
from SSVEPA.utils.common import get_size

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded! with following info: \n{headers}")
                # Verify the file type and size
                if zipfile.is_zipfile(self.config.local_data_file):
                    logger.info("Downloaded file is a valid zip file.")
                else:
                    logger.warning("Downloaded file is not a valid zip file.")
            except Exception as e:
                logger.error(f"Failed to download the file due to: {e}")
                raise RuntimeError(f"Failed to download the file due to: {e}")
        else:
            #file_size = os.path.getsize(self.config.local_data_file)
            file_size = get_size(Path(self.config.local_data_file))
            logger.info(f"File already exists of size: {file_size}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Files extracted to {unzip_path}")

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error("An error occurred during data ingestion.", exc_info=True)
    raise e


[2024-07-09 23:48:31,518: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-09 23:48:31,519: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 23:48:31,520: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 23:48:31,521: INFO: common: created directory at: artifacts]
[2024-07-09 23:48:31,521: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-09 23:48:31,522: INFO: 1779564001: File already exists of size: ~ 23 KB]
[2024-07-09 23:48:31,524: INFO: 1779564001: Files extracted to artifacts/data_ingestion]
